In [119]:
import requests
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd 
import random
import os
import spacy
nlp = spacy.load('en_core_web_sm')

In [140]:
import nltk
import ssl 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import FreqDist
nltk.download("stopwords")
from nltk.corpus import stopwords
import string

from string import punctuation
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mahithaappasani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
def ScrapeText(url,lower_index,upper_index):
    #we are using request package to make a GET request for the website, which means we're getting data from it.
    r=requests.get(url)

    #Setting the correct text encoding of the HTML page
    r.encoding = 'utf-8'

    #Extracting the HTML from the request object
    html = r.text
    
    # Creating a BeautifulSoup object from the HTML
    soup = BeautifulSoup(html)

    # Getting the text out of the soup
    text = soup.get_text()
    
    #getting just the main portion of text
    main_text = text[lower_index:upper_index]
    
    clean_text= main_text.replace("/n", " ")
    
    return clean_text

In [150]:
def getMostCommonWords(clean_text):
    #Splitting data into words and getting meaningful words from text
    words = word_tokenize(clean_text)
    stop_words = set(stopwords.words("english"))
    remove_words = ['\'s','per','  \'']
    stop_words.update(remove_words)

    words_no_punctuation = [word for word in words if word not in punctuation]
    meaningful_words = [word for word in words_no_punctuation if word.casefold() not in stop_words]
    
    #Getting the top 100 most common words
    top_frequency_distribution = FreqDist(meaningful_words).most_common(50)
    top_frequency_df = pd.DataFrame(top_frequency_distribution, columns =['Word', 'Frequency'])
    
    return top_frequency_df

In [157]:
def getMostCommonPhrases(clean_text):
    noun_phrase = list()
    for chunk in nlp(clean_text).noun_chunks:
        new = str(chunk)
        if len(new.split()) != 1:
            noun_phrase.append(new)

    frequency_distribution_np = FreqDist(noun_phrase).most_common(30)    

    noun_phrase_df = pd.DataFrame(frequency_distribution_np, columns =['Noun Phrase', 'Frequency'])
    
    return noun_phrase_df

In [158]:
url_list = ['https://koreanserums.com/american-beauty-standards/',
        'https://tower.mastersny.org/10976/features/the-western-lens-on-beauty-how-european-beauty-standards-are-still-in-play-today/#:~:text=The%20features%20which%20tend%20to,decade%2C%20it%20still%20remains%20dominant.',
           'https://aninjusticemag.com/stop-imposing-eurocentric-beauty-standards-1560f58593d2']

#lower and upper indices for each article to get main text
indices_list=[[562,11835],[1422,8155],[343,7100]]



In [159]:
for i in range(len(url_list)):
    url = url_list[i]
    lower_index = indices_list[i][0]
    upper_index = indices_list[i][1]
    
    cleaned_text = ScrapeText(url,lower_index,upper_index)
    top_words_df = getMostCommonWords(cleaned_text)
    top_phrases_df = getMostCommonPhrases(cleaned_text)
    
    top_words_df.to_excel('Beauty Standards Article ' + str(i+1) + ' Top Words.xlsx')
    top_phrases_df.to_excel('Beauty Standards Article ' + str(i+1) + ' Top Phrases.xlsx')
    
    